In [ ]:
pip install tensorflow

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/drive')

img_height, img_width = 128, 128
batch_size = 8
epochs = 20

train_dir = '/content/drive/My Drive/dataset/train'
val_dir = '/content/drive/My Drive/dataset/validation'

def print_directory_structure(base_dir):
    for root, dirs, files in os.walk(base_dir):
        print(f'{root}: {len(files)} files')

print_directory_structure(train_dir)
print_directory_structure(val_dir)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


print(f'Class indices: {train_generator.class_indices}')
print(f'Class indices: {val_generator.class_indices}')


print(f'Training samples: {train_generator.samples}')
print(f'Validation samples: {val_generator.samples}')


Mounted at /content/drive
/content/drive/My Drive/dataset/train: 0 files
/content/drive/My Drive/dataset/train/happy: 30 files
/content/drive/My Drive/dataset/train/sad: 36 files
/content/drive/My Drive/dataset/validation: 0 files
/content/drive/My Drive/dataset/validation/sad: 20 files
/content/drive/My Drive/dataset/validation/happy: 14 files
Found 66 images belonging to 2 classes.
Found 34 images belonging to 2 classes.
Class indices: {'happy': 0, 'sad': 1}
Class indices: {'happy': 0, 'sad': 1}
Training samples: 66
Validation samples: 34


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation

model = Sequential([
Conv2D(32, (3, 3), input_shape=(img_height, img_width, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.15), #Dropout changed to 0.15

    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=max(1, val_generator.samples // batch_size),
    epochs=epochs
)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 485ms/step - accuracy: 0.7100 - loss: 1.0470 - val_accuracy: 0.4375 - val_loss: 0.9308
Epoch 2/20
1/8 ━━━━━━━━━━━━━━━━━━━━ 2s 400ms/step - accuracy: 1.0000 - loss: 0.0869

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 1.0000 - loss: 0.0869 - val_accuracy: 0.5000 - val_loss: 1.3077
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 496ms/step - accuracy: 0.7858 - loss: 0.7077 - val_accuracy: 0.5938 - val_loss: 0.6828
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7500 - loss: 1.0013 - val_accuracy: 0.5000 - val_loss: 0.5298
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 440ms/step - accuracy: 0.9252 - loss: 0.2339 - val_accuracy: 0.6875 - val_loss: 0.6000
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7500 - loss: 0.5799 - val_accuracy: 0.5000 - val_loss: 0.4812
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 570ms/step - accuracy: 0.9233 - loss: 0.2028 - val_accuracy: 0.6250 - val_loss: 0.6529
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - accuracy: 0.8750 - loss: 0.1780 - val_accuracy: 0.0000e+00 - val_loss: 1.1185
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 445ms/step - accuracy: 0.9378 - loss: 0.1960 - val_accuracy: 0.5625 - val_loss: 0.8576
Epo

In [ ]:
model.save('happy_sad_cnn_model.h5')

loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.6075 - loss: 0.6761
Validation Loss: 0.7073584198951721
Validation Accuracy: 0.5882353186607361
